In [35]:
import pandas as pd 
import numpy as np                     # For mathematical calculations 
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings   # To ignore any warnings 
warnings.filterwarnings("ignore") 

In [100]:
train = pd.read_excel('consumptionstatemon.xls') 

In [101]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64996 entries, 0 to 64995
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   YEAR                                64996 non-null  int64  
 1   MONTH                               64996 non-null  int64  
 2   STATE                               64996 non-null  object 
 3   TYPE OF PRODUCER                    64996 non-null  object 
 4   ENERGY SOURCE              (UNITS)  64996 non-null  object 
 5   CONSUMPTION                         64996 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 3.0+ MB


In [184]:
train['ENERGY SOURCE              (UNITS)'].unique()

array(['Coal (Short Tons)', 'Petroleum (Barrels)', 'Natural Gas (Mcf)',
       'Other Gases (Billion BTU)'], dtype=object)

In [126]:
train.head()

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE (UNITS),CONSUMPTION
0,2001,1,AK,Total Electric Power Industry,Coal (Short Tons),47615.0
1,2001,1,AK,Total Electric Power Industry,Petroleum (Barrels),124998.0
2,2001,1,AK,Total Electric Power Industry,Natural Gas (Mcf),3941267.0
3,2001,1,AK,"Electric Generators, Electric Utilities",Coal (Short Tons),16535.0
4,2001,1,AK,"Electric Generators, Electric Utilities",Petroleum (Barrels),114198.0


In [127]:
x = train.drop(['CONSUMPTION'],axis = 1)
y = train['CONSUMPTION']

In [128]:
x.head()

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE (UNITS)
0,2001,1,AK,Total Electric Power Industry,Coal (Short Tons)
1,2001,1,AK,Total Electric Power Industry,Petroleum (Barrels)
2,2001,1,AK,Total Electric Power Industry,Natural Gas (Mcf)
3,2001,1,AK,"Electric Generators, Electric Utilities",Coal (Short Tons)
4,2001,1,AK,"Electric Generators, Electric Utilities",Petroleum (Barrels)


In [129]:
train.isnull().sum() 

YEAR                                  0
MONTH                                 0
STATE                                 0
TYPE OF PRODUCER                      0
ENERGY SOURCE              (UNITS)    0
CONSUMPTION                           0
dtype: int64

In [158]:
x.shape

(64996, 5)

In [159]:
from sklearn.preprocessing import LabelEncoder

In [160]:
class MultiColumnLabelEncoder(LabelEncoder):
    """
    Wraps sklearn LabelEncoder functionality for use on multiple columns of a
    pandas dataframe.

    """
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, dframe):
        """
        Fit label encoder to pandas columns.

        Access individual column classes via indexig `self.all_classes_`

        Access individual column encoders via indexing
        `self.all_encoders_`
        """
        # if columns are provided, iterate through and get `classes_`
        if self.columns is not None:
            # ndarray to hold LabelEncoder().classes_ for each
            # column; should match the shape of specified `columns`
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            self.all_encoders_ = np.ndarray(shape=self.columns.shape,
                                            dtype=object)
            for idx, column in enumerate(self.columns):
                # fit LabelEncoder to get `classes_` for the column
                le = LabelEncoder()
                le.fit(dframe.loc[:, column].values)
                # append the `classes_` to our ndarray container
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                # append this column's encoder
                self.all_encoders_[idx] = le
        else:
            # no columns specified; assume all are to be encoded
            self.columns = dframe.iloc[:, :].columns
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            for idx, column in enumerate(self.columns):
                le = LabelEncoder()
                le.fit(dframe.loc[:, column].values)
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
        return self

    def fit_transform(self, dframe):
        """
        Fit label encoder and return encoded labels.

        Access individual column classes via indexing
        `self.all_classes_`

        Access individual column encoders via indexing
        `self.all_encoders_`

        Access individual column encoded labels via indexing
        `self.all_labels_`
        """
        # if columns are provided, iterate through and get `classes_`
        if self.columns is not None:
            # ndarray to hold LabelEncoder().classes_ for each
            # column; should match the shape of specified `columns`
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            self.all_encoders_ = np.ndarray(shape=self.columns.shape,
                                            dtype=object)
            self.all_labels_ = np.ndarray(shape=self.columns.shape,
                                          dtype=object)
            for idx, column in enumerate(self.columns):
                # instantiate LabelEncoder
                le = LabelEncoder()
                # fit and transform labels in the column
                dframe.loc[:, column] =\
                    le.fit_transform(dframe.loc[:, column].values)
                # append the `classes_` to our ndarray container
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
                self.all_labels_[idx] = le
        else:
            # no columns specified; assume all are to be encoded
            self.columns = dframe.iloc[:, :].columns
            self.all_classes_ = np.ndarray(shape=self.columns.shape,
                                           dtype=object)
            for idx, column in enumerate(self.columns):
                le = LabelEncoder()
                dframe.loc[:, column] = le.fit_transform(
                        dframe.loc[:, column].values)
                self.all_classes_[idx] = (column,
                                          np.array(le.classes_.tolist(),
                                                  dtype=object))
                self.all_encoders_[idx] = le
        return dframe.loc[:, self.columns].values

    def transform(self, dframe):
        """
        Transform labels to normalized encoding.
        """
        if self.columns is not None:
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[
                    idx].transform(dframe.loc[:, column].values)
        else:
            self.columns = dframe.iloc[:, :].columns
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .transform(dframe.loc[:, column].values)
        return dframe.loc[:, self.columns].values

    def inverse_transform(self, dframe):
        """
        Transform labels back to original encoding.
        """
        if self.columns is not None:
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .inverse_transform(dframe.loc[:, column].values)
        else:
            self.columns = dframe.iloc[:, :].columns
            for idx, column in enumerate(self.columns):
                dframe.loc[:, column] = self.all_encoders_[idx]\
                    .inverse_transform(dframe.loc[:, column].values)
        return dframe.loc[:, self.columns].values

In [161]:
x1 = x.iloc[:, :].select_dtypes(include=['object']).columns
mcle = MultiColumnLabelEncoder(columns=x1)
mcle

MultiColumnLabelEncoder(columns=Index([], dtype='object'))

In [162]:
mcle.fit(x)

MultiColumnLabelEncoder(columns=Index([], dtype='object'))

In [163]:
x1 = mcle.transform(x)

In [164]:
import pickle 

file = open('label_encodings', 'wb')

# dump information to that file
pickle.dump(mcle, file)

# close the file
file.close()

In [165]:
encoded_features = np.concatenate((x.select_dtypes(['int64']).values,x1),axis=1)
encoded_features

array([[2.001e+03, 1.000e+00, 0.000e+00, 5.000e+00, 0.000e+00],
       [2.001e+03, 1.000e+00, 0.000e+00, 5.000e+00, 3.000e+00],
       [2.001e+03, 1.000e+00, 0.000e+00, 5.000e+00, 1.000e+00],
       ...,
       [2.007e+03, 1.200e+01, 4.400e+01, 2.000e+00, 3.000e+00],
       [2.007e+03, 1.200e+01, 4.400e+01, 2.000e+00, 1.000e+00],
       [2.007e+03, 1.200e+01, 4.400e+01, 2.000e+00, 2.000e+00]])

In [166]:
encoded_features.shape

(64996, 5)

In [167]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(encoded_features,y,test_size=.30)

In [168]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
clf = RandomForestRegressor() 

In [169]:
param = {'bootstrap': [True, False],
'max_depth': [5,10, None],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [50,100]} 

In [170]:
grid_search = GridSearchCV(clf,param_grid = param,cv = 5) 

In [171]:
#random_search = RandomizedSearchCV(clf, param_distributions=pram2,n_iter=10, cv=5) 
random_search = RandomizedSearchCV(clf, param_distributions=param,n_iter=10, cv=5) 

In [172]:
regr = RandomForestRegressor(bootstrap= True,
 max_depth= None,
 min_samples_leaf= 1,
 min_samples_split= 2,
 n_estimators= 20)

In [173]:
regr.fit(x_train,y_train) 

RandomForestRegressor(n_estimators=20)

In [174]:
print(x_train.shape)
print(x_test.shape)

(45497, 5)
(19499, 5)


In [175]:
file = open('random_forest_model', 'wb')
pickle.dump(regr, file)
file.close()
loaded_model1 = pickle.load(open('random_forest_model', 'rb'))
result = loaded_model1.predict(x_test)
print(result) 

[7.67010450e+03 1.20842893e+07 3.10000000e+00 ... 3.96666000e+02
 1.22500000e+01 3.94610500e+04]


In [176]:
regr.score(x_train,y_train) ,regr.score(x_test,y_test) 

(0.9984751272102944, 0.9821990274557068)

In [177]:
pred = regr.predict(x_test)

In [178]:
pred[5:10] 

array([  45672.797,  835652.708, 1934214.85 ,       0.   ,  515950.043])

In [179]:
pred1 = regr.predict(np.array([[ 425.6, 8070.5828, 320005.5162,   4557.7218,   9870.62]])) 
pred1

array([5490.436])